In [ ]:
from http import client
import rospy
import sys, select, termios, tty
import os
import numpy as np
import signal


import actionlib
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseActionGoal, MoveBaseActionResult
from actionlib_msgs.msg import GoalID, GoalStatusArray
from final_assignment.srv import ChangeOp, ChangeOpResponse
from std_msgs.msg import String, Float64, Bool, Float32
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from tf.transformations import quaternion_matrix


import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
import jupyros as jr
import math
import actionlib
import actionlib.msg
import tf

from matplotlib.animation import FuncAnimation
from ipywidgets import Button,Layout,ButtonStyle,GridBox,interact,interactive,fixed,interact_manual, VBox, HBox
from IPython.display import display, clear_output
import functools

import jupyros as jr



%matplotlib widget 
#magic function that allows the user to interact with the plots 

rospy.init_node('new_user_interface')

change_op=rospy.ServiceProxy('/change_operation',ChangeOp)

manual_control_pub = rospy.Publisher('/suggestion',Twist,queue_size=1,latch=True) 

go_to_target_pub = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size = 1)
cancel_goal_pub = rospy.Publisher("/move_base/cancel", GoalID, queue_size = 1)

desired_vel = Twist()
coeff_l = 1.0 
coeff_a = 1.0 
master_speed=0.8

sent_goal=False








In [ ]:
# Change Operation buttons:

b1 = widgets.Button(description='Reach Target Coordinates',
                    disabled=False,
                    tooltip='Switch to Reach Target Coordinates',
                    style=ButtonStyle(button_color='Moccasin'),
                    layout=Layout(width='auto', grid_area='b1')
                    )
b2 = widgets.Button(description='Manual Control',
                    disabled=False,
                    tooltip='Switch to manual control of the rover',
                    style=ButtonStyle(button_color='Moccasin'),
                    layout=Layout(width='auto', grid_area='b2')
                    )
b3 = widgets.Button(description='Manual Control with Collision Assist',
                    disabled=False,
                    tooltip='Switch to Manual Control with Collision Assist',
                    style=ButtonStyle(button_color='Moccasin'),
                    layout=Layout(width='auto', grid_area='b3')

                    )

# Change Velocity buttons:
b4 = widgets.Button(disabled=False,
                    tooltip='Move forward',
                    icon='fa-arrow-circle-up',
                    style=ButtonStyle(button_color='DodgerBlue'),
                    layout=Layout(width='auto', grid_area='b4'))

b5 = widgets.Button(disabled=False,
                    tooltip='Move backwards',
                    icon='fa-arrow-circle-down',
                    style=ButtonStyle(button_color='DodgerBlue'),
                    layout=Layout(width='auto', grid_area='b5'))

b6 = widgets.Button(  
    disabled=False,
    tooltip='Move to the left',
    icon='fa-arrow-circle-left',
    style=ButtonStyle(button_color='DodgerBlue'),
    layout=Layout(width='auto', grid_area='b6'))

b7 = widgets.Button(  
    disabled=False,
    tooltip='Move to the right',
    icon='fa-arrow-circle-right',
    style=ButtonStyle(button_color='DodgerBlue'),
    layout=Layout(width='auto', grid_area='b7'))

b8 = widgets.Button(description='Stop',
                    disabled=False,
                    tooltip='Stop rover ',
                    style=ButtonStyle(button_color='LightBlue'),
                    layout=Layout(width='auto', grid_area='b8'))

# menu and target buttons


b9 = widgets.Button(description='Send goal',
                    disabled=False,
                    tooltip='Send the goal position',
                    style=ButtonStyle(button_color='Khaki'),
                    layout=Layout(width='auto', grid_area='b9'))

b10 = widgets.Button(description='cancel goal',
                     disabled=False,
                     tooltip='Cancel the goal sent to the move_base action server',
                     icon='fa-undo',
                     style=ButtonStyle(button_color='Goldenrod'),
                     layout=Layout(width='auto', grid_area='b10'))

b11 = widgets.Button(description='Shut down simulation',
                    disabled=False,
                    tooltip='Send shutdown signal to ROS',
                    style=ButtonStyle(button_color='Red'),
                    layout=Layout(width='auto', grid_area='b11'))

# float selction tabs
ft1 = widgets.FloatText(value=0.0,
                        description='x',
                        disabled=False,
                        layout=Layout(width='auto', grid_area='ft1'))

ft2 = widgets.FloatText(value=0.0,
                        description='y',
                        disabled=False,
                        layout=Layout(width='auto', grid_area='ft2'))

# velocity sliders
fs1 = widgets.FloatSlider(value=1.0,
                          min=0.1,
                          max=2.0,
                          step=0.1,
                          description='Linear  ',
                          disabled=False,
                          continuous_update=False,
                          orientation='vertical',
                          readout=True,
                          readout_format='.1f',
                          grid_area='fs1')

fs2 = widgets.FloatSlider(value=1.0,
                          min=0.1,
                          max=2.0,
                          step=0.1,
                          description='Angular',
                          disabled=False,
                          continuous_update=False,
                          orientation='vertical',
                          readout=True,
                          readout_format='.1f',
                          grid_area='fs2')

# reorganizing buttons into a command console

q5 = GridBox(children=[b4, b5, b6, b7, b8],
             layout=Layout(width='80%',
                           height='100%',
                           grid_template_areas='''
                                     " . b4 . "
                                     " b6 b8 b7 "
                                     " . b5 . "
                                     '''))

q0 = GridBox(children=[b9, ft1, ft2, b10, b11],
             layout=Layout(
             width='100%',
             height='100%',
             grid_template_areas='''
                                " b9 "
                                " ft1 "
                                " ft2 "
                                " b10 "
                                " b11 "
                                '''))

q1 = GridBox(children=[b1, b2, b3],
             layout=Layout(
             width='100%',
             height='100%',
             grid_template_areas='''
                                "b1 b2 b3"
                                '''))

q2 = HBox([fs1, fs2, q5])

q3 = GridBox(children=[q1, q2, q0],
             layout=Layout(
             width='100%',
             height='100%',
             grid_template_areas='''
                                "q1"
                                "q2"
                                "q0"
                                '''))


In [ ]:
# Button Functions
def change_operation(b,rs_):
    
    rospy.wait_for_service('/change_operation')    
    try:
        #send request through the server client
        change_op(rs_)       

    except rospy.ServiceException as e:
        print("Couldn't execute service call: %s" %e)                 
                
def change_vel(b,rs_='stop'): 
    
    global desired_vel
    global coeff_l
    global coeff_a
    global master_speed
    
    if(rs_=='front'):
        desired_vel.linear.x=coeff_l*master_speed
    elif(rs_=='back'):
        desired_vel.linear.x=-coeff_l*master_speed
    elif(rs_=='left'):
        desired_vel.angular.z=coeff_a*master_speed
    elif(rs_=='right'):
        desired_vel.angular.z=-coeff_a*master_speed
    elif(rs_=='stop'):
        desired_vel.linear.x=coeff_l*0.0
        desired_vel.angular.z=coeff_a*0.0
    
    #publish resulting twist message on the /suggestion topic
    manual_control_pub.publish(desired_vel)


# Functions for the sliders:

def update_linear(change): 
    
    global coeff_l
    global desired_vel
    
    desired_vel.linear.x = desired_vel.linear.x/coeff_l*fs1.value 
    manual_control_pub.publish(desired_vel)

def update_angular(change):
    
    global coeff_a
    global required_vel
    desired_vel.angular.z = desired_vel.angular.z/coeff_a*fs2.value
    manual_control_pub.publish(desired_vel) 

def send_goal(b):

    my_goal = MoveBaseActionGoal()
    my_goal.goal.target_pose.pose.position.x = ft1.value
    my_goal.goal.target_pose.pose.position.y = ft2.value
    my_goal.goal.target_pose.header.frame_id = "map"
    my_goal.goal.target_pose.pose.orientation.w = 1
    go_to_target_pub.publish(my_goal)

def cancel_goal(b):
    
    cancel_id = GoalID()
    cancel_goal_pub.publish(cancel_id)
    
def shutdown_sim(b):

    # Shutdown all ROS nodes
    os.system("rosnode kill -a")

    # Wait for a short time to allow nodes to shut down
    rospy.sleep(2)

    # Terminate the ROS Master (roscore)
    os.kill(os.getpid(), signal.SIGINT)

In [ ]:
# Linking Buttons to their functions

# select operation commands
b1.on_click(functools.partial(change_operation, rs_='t'))
b2.on_click(functools.partial(change_operation, rs_='g'))
b3.on_click(functools.partial(change_operation, rs_='b'))


b9.on_click(send_goal)
b10.on_click(cancel_goal)
b11.on_click(shutdown_sim)

# directional commands
b4.on_click(functools.partial(change_vel, rs_='front'))
b5.on_click(functools.partial(change_vel, rs_='back'))
b6.on_click(functools.partial(change_vel, rs_='left'))
b7.on_click(functools.partial(change_vel, rs_='right'))
b8.on_click(functools.partial(change_vel, rs_='stop'))

# Linking Sliders to their functions

fs1.observe(update_linear, names='value')
fs2.observe(update_angular, names='value')

In [ ]:
class Visualiser:
    def __init__(self): 
                
        # initialize variables for the first graph:
        self.odom, self.odom_axes = plt.subplots()
        self.ln1, = self.odom_axes.plot([], [], color='green', marker='H', markersize=13, linewidth=0)
        self.x_data, self.y_data = [] , [] 

        #initialize variables for the second graph
        self.scan, self.scan_axes = plt.subplots()
        self.ln2, = self.scan_axes.plot([], [], color='red', marker='o', markersize=3, linewidth=0)
        self.scan_x_data_rf, self.scan_y_data_rf = [] , [] 
        self.scan_z_data_rf=[0]*720 
        self.filler=[1]*720
        
        self.pos_array_rf = np.empty((4,720)) 
        self.pos_array_bf = np.empty((4,720)) 
        self.complete_pos_array_bf = np.empty((4,720))
        self.t_matrix = np.identity(4) 
        self.complete_t_matrix = np.identity(4)
        self.angle = -math.pi/2
        self.i = 0
        self.j = 0 


        #initialize variables for the third graph:
        self.goals, self.goals_axes = plt.subplots()
        self.rects = self.goals_axes.bar([2,4,6,8],
                                         [0,0,0,0],
                                         align='center',
                                         width=0.3,
                                         color=['yellow',
                                                'green',
                                                'blue',
                                                'pink'])
        self.canceled_goals = 0
        self.reached_goals = 0 
        self.aborted_goals = 0
        self.preempted_goals = 0
        self.goals_array = np.empty((4,1)) 

        
    # First graph functions: Robot position
    # -----------------------------------------------------------------------------------------------------    
    def plot_init1(self): 

        self.ln1.set_data([],[]) 
        self.odom_axes.set_xlim(-10, 10) 
        self.odom_axes.set_ylim(-10, 10)
        self.odom_axes.set_title("Robot Position") 
        self.odom_axes.set_xlabel("x") 
        self.odom_axes.set_ylabel("y") 
        return self.ln1 
    
    def clbk_odom(self, msg):

        self.x_data.clear()
        self.y_data.clear()
       
        self.x_data.append(msg.pose.pose.position.x) 
        self.y_data.append(msg.pose.pose.position.y) 
        self.t_matrix = np.identity(4)

        self.t_matrix = np.array(quaternion_matrix([msg.pose.pose.orientation.x,
                                            msg.pose.pose.orientation.y,
                                            msg.pose.pose.orientation.z,
                                            msg.pose.pose.orientation.w]))
        self.t_matrix[0,3]= msg.pose.pose.position.x
        self.t_matrix[1,3]= msg.pose.pose.position.y
        self.t_matrix[2,3]= msg.pose.pose.position.z
        
        self.complete_t_matrix=self.t_matrix 

    def update_plot1(self, frame):
        self.ln1.set_data(self.x_data, self.y_data)
        return self.ln1
    

    # Second Graph functions: LIDAR readings
    # -----------------------------------------------------------------------------------------------------
    def plot_init2(self):
        self.ln2.set_data([],[])
        self.scan_axes.set_xlim(-10, 10)
        self.scan_axes.set_ylim(-10, 10)
        self.scan_axes.set_title("LIDAR Scan")
        self.scan_axes.set_xlabel("x")
        self.scan_axes.set_ylabel("y")
        return self.ln2
    
    def clbk_scan(self, msg):
        self.step_size = math.pi/len(msg.ranges) 
        self.angle = -math.pi/2
        self.scan_x_data_rf.clear()
        self.scan_y_data_rf.clear()

        self.pos_array_rf = np.empty((4,720))
        self.pos_array_bf = np.empty((4,720))
        self.j = self.j+1
        
        if(self.j==15):
           
            for r in msg.ranges:
               
                self.scan_x_data_rf.append(r*math.cos(self.angle)) 
                
                self.scan_y_data_rf.append(r*math.sin(self.angle))
                
                self.angle += self.step_size
           
            self.pos_array_rf = np.vstack((self.scan_x_data_rf, self.scan_y_data_rf, self.scan_z_data_rf, self.filler))

            for r, index in enumerate(msg.ranges):
                self.pos_array_bf[:, index] = np.dot(self.complete_t_matrix, self.pos_array_rf[:, index])
            self.complete_pos_array_bf = self.pos_array_bf
            self.j = 0 
       
    def update_plot2(self, frame): 
        self.ln2.set_data(list(self.complete_pos_array_bf[0,:]), list(self.complete_pos_array_bf[1,:]))
        return self.ln2
    
    
    # Third Graph functions: Reached Goals
    # -----------------------------------------------------------------------------------------------------
    def plot_init3(self): 
        self.goals_axes.set_xlim(0, 10)
        self.goals_axes.set_ylim(0, 10)
        self.goals_axes.set_title("Goals")
        self.goals_axes.set_ylabel("")
        self.goals_axes.set_xticks([2,4,6,8])
        self.goals_axes.set_xticklabels(['canceled', 'reached', 'aborted','preempted'])
        return self.rects 

    def clbk_goalstatus(self,msg):
        status = msg.status.status
        
        if status==2:
            self.canceled_goals += 1
        elif status==3:
            self.reached_goals += 1
        elif status==4:
            self.aborted_goals += 1
        elif status==5:
            self.preempted_goals += 1
            

    def update_plot3(self, frame): 
        self.goals_array = np.array([self.canceled_goals, self.reached_goals, self.aborted_goals, self.preempted_goals]).T
        for self.rect, self.goal in zip(self.rects, self.goals_array):
            self.rect.set_height(self.goal)
        return self.rects


   


In [ ]:
display(q3)

In [ ]:
vis = Visualiser() 
sub_odom = rospy.Subscriber('/odom', Odometry, vis.clbk_odom)
sub_scan = rospy.Subscriber('/scan', LaserScan, vis.clbk_scan) 
sub_goalstatus = rospy.Subscriber('/move_base/result', MoveBaseActionResult , vis.clbk_goalstatus)
animation = FuncAnimation(vis.odom, vis.update_plot1, init_func=vis.plot_init1, frames=100, interval=200, blit=True) 
animation2 = FuncAnimation(vis.scan, vis.update_plot2, init_func=vis.plot_init2, frames=100, interval=200, blit=True)
animation3 = FuncAnimation(vis.goals, vis.update_plot3, init_func=vis.plot_init3, frames=100, interval=100, blit=True)

plt.show(block=True) 